# File for applying time warping to .wav files

In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

import librosa
import librosa.display

In [ ]:
#NB! Set your audio folder here before using this file
AUDIO_FOLDER = "audio files"
audio_files = [join(AUDIO_FOLDER, f) for f in listdir(AUDIO_FOLDER) if isfile(join(AUDIO_FOLDER,
                                                                                   f))]
print(audio_files)

In [ ]:
#Sanity check, can be skipped

#MP3 does not work!
#https://github.com/librosa/librosa/issues/945

#x_1, fs = librosa.load('./audio/short files/clavichord.WAV')
for file_name in audio_files:
    print(file_name)
    x_1, fs = librosa.load(file_name)
    #print("The time series for this file is", x_1[:10])
    print("The length of the time series is", len(x_1))
    print("The sampling rate of the file is", fs)
    print()

In [ ]:

x = audio_files[0]
x_1, fs = librosa.load(x)

In [ ]:
#How to save files

import soundfile
soundfile.write("test.wav", x_1, fs)

# DTW to stretch music

In [ ]:

#NB! Change these two as you desire
f1 = audio_files[3] #File you want as your baseline
f2 = audio_files[-2] #File you want to warp

x_1, fs = librosa.load(f1)
x_2, fs = librosa.load(f2)

In [ ]:
#Experiment with this as you wish. This is the hop-length for chromagrams. 
#Really short ones result in very high memory usage.
hl = 25

In [ ]:
x_1_chroma = librosa.feature.chroma_stft(y=x_1, sr=fs, tuning=0, norm=2, hop_length = hl)
x_2_chroma = librosa.feature.chroma_stft(y=x_2, sr=fs, tuning=0, norm=2, hop_length = hl)

In [ ]:
x_1.shape

In [ ]:

#time warping
D, wp = librosa.sequence.dtw(X=x_1_chroma, Y=x_2_chroma, metric='cosine')


In [ ]:
#Getting the matching indices (multiplied by hl to get back to original scale of our series)

s1 = np.array(list(reversed([x[0] for x in wp])))*hl
s2 = np.array(list(reversed([x[1] for x in wp])))*hl

In [ ]:
x_2_n = [x_2[0]]

In [ ]:
#Stretching to make x_2 the same length as x_1 by aligning the different parts

for i in range(len(s1)-1):
    
    r1 = (s1[i+1]-s1[i])
    
    #If multiple parts of x_2 match to single point in x_2, we remove that part of the song
    if r1 == 0:
        continue
    r2 = (s2[i+1]-s2[i])
    
    #If multiple parts of x_1 match to a single point in x_2, we just continue that
    if r2 == 0:
        x_2_n.extend([x_2_n[-1] for i in range(r1)])
        continue
        
    r = r1/r2

    old = x_2[s2[i]+1:s2[i+1]+1]
    new = librosa.effects.time_stretch(old, r)
    
    #print(len(old), len(new), r)
    x_2_n.extend(new)
    
    

In [ ]:
soundfile.write("warped.wav", x_2_n, fs)